<a href="https://colab.research.google.com/github/0123-hamn/BCT_Project/blob/main/BCT_Email_Writter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn pandas nltk

In [2]:
import pandas as pd
import nltk
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
data = {
    "purpose": [
        "Request leave for 2 days",
        "Apply for job position",
        "Pitch new product to client",
        "Schedule meeting with manager",
        "Follow up on payment",
        "Request internship opportunity",
        "Propose business collaboration",
        "Complaint about service issue"
    ],
    "tone": [
        "formal",
        "formal",
        "persuasive",
        "formal",
        "semi-formal",
        "formal",
        "persuasive",
        "semi-formal"
    ]
}

df = pd.DataFrame(data)
df

,purpose,tone
0,Request leave for 2 days,formal
1,Apply for job position,formal
2,Pitch new product to client,persuasive
3,Schedule meeting with manager,formal
4,Follow up on payment,semi-formal
5,Request internship opportunity,formal
6,Propose business collaboration,persuasive
7,Complaint about service issue,semi-formal


In [5]:
label_encoder = LabelEncoder()
df['tone_encoded'] = label_encoder.fit_transform(df['tone'])
df

,purpose,tone,tone_encoded
0,Request leave for 2 days,formal,0
1,Apply for job position,formal,0
2,Pitch new product to client,persuasive,1
3,Schedule meeting with manager,formal,0
4,Follow up on payment,semi-formal,2
5,Request internship opportunity,formal,0
6,Propose business collaboration,persuasive,1
7,Complaint about service issue,semi-formal,2


In [6]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df['purpose'])
y = df['tone_encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [7]:
templates = {
    "formal": """Dear {recipient},

I hope this email finds you well.

I am writing to formally request {purpose}.

Please let me know if any further information is required.

Sincerely,
{sender}
""",

    "semi-formal": """Hi {recipient},

I hope you're doing well.

I wanted to talk about {purpose}. Please let me know your thoughts.

Best regards,
{sender}
""",

    "persuasive": """Dear {recipient},

I am excited to share {purpose} with you.

I strongly believe this will bring significant value and would love the opportunity to discuss further.

Looking forward to your response.

Best regards,
{sender}
"""
}

In [8]:
def generate_subject(purpose):
    keywords = purpose.split()
    main_words = " ".join(keywords[:3])
    return f"Regarding {main_words}"

In [9]:
def generate_email(purpose, recipient, sender, tone=None):

    # If tone not provided → predict using ML
    if tone is None:
        vector = vectorizer.transform([purpose])
        predicted = model.predict(vector)
        tone = label_encoder.inverse_transform(predicted)[0]

    template = templates[tone]

    email_body = template.format(
        recipient=recipient,
        sender=sender,
        purpose=purpose
    )

    subject = generate_subject(purpose)

    return subject, email_body, tone

In [10]:
purpose_input = input("Enter email purpose: ")
recipient_name = input("Enter recipient name: ")
sender_name = input("Enter sender name: ")
tone_input = input("Enter tone (formal/semi-formal/persuasive) or press Enter for auto: ")

if tone_input.strip() == "":
    tone_input = None

subject, email, detected_tone = generate_email(
    purpose_input,
    recipient_name,
    sender_name,
    tone_input
)

print("\nDetected Tone:", detected_tone)
print("\nSubject:", subject)
print("\nGenerated Email:\n")
print(email)

Enter email purpose: Invitation to a birthday party
Enter recipient name: Parna Mandal
Enter sender name: Hemanta Ghosh
Enter tone (formal/semi-formal/persuasive) or press Enter for auto: formal

Detected Tone: formal

Subject: Regarding Invitation to a

Generated Email:

Dear Parna Mandal,

I hope this email finds you well.

I am writing to formally request Invitation to a birthday party.

Please let me know if any further information is required.

Sincerely,
Hemanta Ghosh

